# Etude de marché pour le compte de l'entreprise "La poule qui chante" partie 1 

Le but de ce projet sera d'effectuer une étude de marché dans l'optique du developpement de l'entreprise à l'international.
Ce notebook est le premier d'une série de 2 notebooks.

# Préparation, nettoyage, et analyse exploratoire des données 

## Import des librairies

In [351]:
import pandas as pd
import numpy
from IPython.display import Image
import plotly.express as px
import plotly.graph_objs as go

## Import des données

In [352]:
df_dispo_alim_2017 = pd.read_csv("DisponibiliteAlimentaire_2017.csv")
df_pop_2000_2018 = pd.read_csv("Population_2000_2018.csv")
df_pib_stab_pop_2017 = pd.read_csv("PIB_Stabilite_Politique_2017.csv")
df_internet_taux = pd.read_csv("internet_taux_pays.csv")
df_revenu_taxe_commerce = pd.read_csv("revenu_taxe_ci.csv")
df_distance = pd.read_csv("distances.csv", index_col=0)
df_iso = pd.read_csv("fr-3166.csv")

## Cadre d'analyse macroéconomique : PESTEL

Nous avons choisi de se baser sur le cadre d'analyse "PESTEL" pour réaliser cette étude de marché.
Voici le cadre d'analyse ainsi que des exemples des variables que chaque composante pourrait contenir : 



![cadre d'analyse pestel](pestel.png)

### Choix des variables pour le PESTEL

- Politique : indice de stabilité politique (source: FAO)
- Economie : PIB par habitant (source: FAO)
- Social : consommation de volailles , population (source: FAO)
- Technologie : population ayant accès à internet (source: World Bank)
- Environnement : distance depuis la France (source: https://github.com/mlschneid/country-distance/blob/master/output/distance-matrix.csv )
- Légal : pourcentage de revenu issu des taxes sur le commerce (source: World Bank)

## DF dispo_alim

In [353]:
# On affiche les 5 première lignes du DF

df_dispo_alim_2017.head()

,Code Domaine,Domaine,Code zone,Zone,Code Élément,Élément,Code Produit,Produit,Code année,Année,Unité,Valeur,Symbole,Description du Symbole
0,FBS,Nouveaux Bilans Alimentaire,2,Afghanistan,5511,Production,2511,Blé et produits,2017,2017,Milliers de tonnes,4281.0,S,Données standardisées
1,FBS,Nouveaux Bilans Alimentaire,2,Afghanistan,5611,Importations - Quantité,2511,Blé et produits,2017,2017,Milliers de tonnes,2302.0,S,Données standardisées
2,FBS,Nouveaux Bilans Alimentaire,2,Afghanistan,5072,Variation de stock,2511,Blé et produits,2017,2017,Milliers de tonnes,-119.0,S,Données standardisées
3,FBS,Nouveaux Bilans Alimentaire,2,Afghanistan,5911,Exportations - Quantité,2511,Blé et produits,2017,2017,Milliers de tonnes,0.0,S,Données standardisées
4,FBS,Nouveaux Bilans Alimentaire,2,Afghanistan,5301,Disponibilité intérieure,2511,Blé et produits,2017,2017,Milliers de tonnes,6701.0,S,Données standardisées


In [354]:
# On vérifie les données avec le méthode info

df_dispo_alim_2017.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 176600 entries, 0 to 176599
Data columns (total 14 columns):
 #   Column                  Non-Null Count   Dtype  
---  ------                  --------------   -----  
 0   Code Domaine            176600 non-null  object 
 1   Domaine                 176600 non-null  object 
 2   Code zone               176600 non-null  int64  
 3   Zone                    176600 non-null  object 
 4   Code Élément            176600 non-null  int64  
 5   Élément                 176600 non-null  object 
 6   Code Produit            176600 non-null  int64  
 7   Produit                 176600 non-null  object 
 8   Code année              176600 non-null  int64  
 9   Année                   176600 non-null  int64  
 10  Unité                   176600 non-null  object 
 11  Valeur                  176600 non-null  float64
 12  Symbole                 176600 non-null  object 
 13  Description du Symbole  176600 non-null  object 
dtypes: float64(1), int64

Les data types ont l'air de concorder et il n'y a pas de valeurs nulles

In [355]:
# On check la présence de doublons

df_dispo_alim_2017.duplicated().any()

False

Il n'y a pas de doublon

In [356]:
# On nettoie le DF en supprimant les colonnes inutiles à notre analyse

df_dispo_alim_2017 = df_dispo_alim_2017.drop(columns=[
    "Code Domaine", "Domaine", "Code zone", "Code Élément", "Code Produit",
    "Code année", "Symbole", "Description du Symbole"
])

In [357]:
# On crée un pivot table qui contient la disponibilité alimentaire intérieure pour chaque pays

total_dispo_alim = df_dispo_alim_2017.loc[df_dispo_alim_2017["Élément"] ==
                                       "Disponibilité intérieure"]
pivot_total_dispo_alim = total_dispo_alim.pivot_table(
    index="Zone", values="Valeur", aggfunc={"Valeur": numpy.sum})
pivot_total_dispo_alim = pivot_total_dispo_alim.rename(
    columns={"Valeur": "Total disponibilité intérieure"})
pivot_total_dispo_alim

,Total disponibilité intérieure
Zone,
Afghanistan,15139.0
Afrique du Sud,66840.0
Albanie,4879.0
Algérie,45365.0
Allemagne,174960.0
...,...
Émirats arabes unis,11795.0
Équateur,23810.0
États-Unis d'Amérique,849821.0


In [358]:
# On crée un pivot table qui contient la disponibilité alimentaire intérieure en volaille

part_poulet_dispo_alim = df_dispo_alim_2017.loc[
    (df_dispo_alim_2017["Élément"] == "Disponibilité intérieure")
    & (df_dispo_alim_2017["Produit"] == "Viande de Volailles")]

pivot_part_poulet_dispo_alim = part_poulet_dispo_alim.pivot_table(
    index="Zone", values="Valeur", aggfunc={"Valeur": numpy.sum})

pivot_part_poulet_dispo_alim = pivot_part_poulet_dispo_alim.rename(
    columns={"Valeur": "Poulet dispo intérieur"})

pivot_part_poulet_dispo_alim

,Poulet dispo intérieur
Zone,
Afghanistan,57.0
Afrique du Sud,2118.0
Albanie,47.0
Algérie,277.0
Allemagne,1739.0
...,...
Émirats arabes unis,412.0
Équateur,341.0
États-Unis d'Amérique,18266.0


In [359]:
# On calcule la part de dispo intérieure en volaille en pourcentage

calcul_part_poulet_dispo = pd.merge(pivot_total_dispo_alim,
                                    pivot_part_poulet_dispo_alim,
                                    on="Zone",
                                    how="left")

calcul_part_poulet_dispo["% Volaille / Dispo intérieur"] = round(
    (calcul_part_poulet_dispo["Poulet dispo intérieur"] /
     calcul_part_poulet_dispo["Total disponibilité intérieure"]) * 100, 2)

calcul_part_poulet_dispo

,Total disponibilité intérieure,Poulet dispo intérieur,% Volaille / Dispo intérieur
Zone,,,
Afghanistan,15139.0,57.0,0.38
Afrique du Sud,66840.0,2118.0,3.17
Albanie,4879.0,47.0,0.96
Algérie,45365.0,277.0,0.61
Allemagne,174960.0,1739.0,0.99
...,...,...,...
Émirats arabes unis,11795.0,412.0,3.49
Équateur,23810.0,341.0,1.43
États-Unis d'Amérique,849821.0,18266.0,2.15


In [360]:
# On calcule la part de volaille dans la production

# Production totale

total_prod = df_dispo_alim_2017.loc[df_dispo_alim_2017["Élément"] == "Production"]
pivot_total_prod = total_prod.pivot_table(index="Zone",
                                          values="Valeur",
                                          aggfunc={"Valeur": numpy.sum})
pivot_total_prod = pivot_total_prod.rename(
    columns={"Valeur": "Total production"})

# Production de volailles

part_poulet_prod = df_dispo_alim_2017.loc[
    (df_dispo_alim_2017["Élément"] == "Production")
    & (df_dispo_alim_2017["Produit"] == "Viande de Volailles")]
pivot_part_poulet_prod = part_poulet_prod.pivot_table(
    index="Zone", values="Valeur", aggfunc={"Valeur": numpy.sum})
pivot_part_poulet_prod = pivot_part_poulet_prod.rename(
    columns={"Valeur": "Poulet production"})

# Part volailles production

calcul_part_poulet_prod = pd.merge(pivot_total_prod,
                                   pivot_part_poulet_prod,
                                   on="Zone",
                                   how="left")
calcul_part_poulet_prod["% Volaille / Production"] = round(
    (calcul_part_poulet_prod["Poulet production"] /
     calcul_part_poulet_prod["Total production"]) * 100, 2)

calcul_part_poulet_prod

,Total production,Poulet production,% Volaille / Production
Zone,,,
Afghanistan,11609.0,28.0,0.24
Afrique du Sud,68455.0,1667.0,2.44
Albanie,4231.0,13.0,0.31
Algérie,26763.0,275.0,1.03
Allemagne,167828.0,1514.0,0.90
...,...,...,...
Émirats arabes unis,1793.0,48.0,2.68
Équateur,30843.0,340.0,1.10
États-Unis d'Amérique,974034.0,21914.0,2.25


In [361]:
# On réunit disponibilité et production

calcul_part_dispo_prod =pd.merge(calcul_part_poulet_dispo, 
                        calcul_part_poulet_prod,
                        on='Zone',
                        how='outer'
                        )

calcul_part_dispo_prod


,Total disponibilité intérieure,Poulet dispo intérieur,% Volaille / Dispo intérieur,Total production,Poulet production,% Volaille / Production
Zone,,,,,,
Afghanistan,15139.0,57.0,0.38,11609.0,28.0,0.24
Afrique du Sud,66840.0,2118.0,3.17,68455.0,1667.0,2.44
Albanie,4879.0,47.0,0.96,4231.0,13.0,0.31
Algérie,45365.0,277.0,0.61,26763.0,275.0,1.03
Allemagne,174960.0,1739.0,0.99,167828.0,1514.0,0.90
...,...,...,...,...,...,...
Émirats arabes unis,11795.0,412.0,3.49,1793.0,48.0,2.68
Équateur,23810.0,341.0,1.43,30843.0,340.0,1.10
États-Unis d'Amérique,849821.0,18266.0,2.15,974034.0,21914.0,2.25


In [362]:
# On calcule le total nourriture

total_nourriture = df_dispo_alim_2017.loc[df_dispo_alim_2017["Élément"] ==
                                       "Nourriture"]
pivot_total_nourriture = total_nourriture.pivot_table(
    index="Zone", values="Valeur", aggfunc={"Valeur": numpy.sum})
pivot_total_nourriture = pivot_total_nourriture.rename(
    columns={"Valeur": "Total nourriture"})

# Nourriture volailles

part_poulet_nourriture = df_dispo_alim_2017.loc[
    (df_dispo_alim_2017["Élément"] == "Nourriture")
    & (df_dispo_alim_2017["Produit"] == "Viande de Volailles")]
pivot_part_poulet_nourriture = part_poulet_nourriture.pivot_table(
    index="Zone", values="Valeur", aggfunc={"Valeur": numpy.sum})
pivot_part_poulet_nourriture = pivot_part_poulet_nourriture.rename(
    columns={"Valeur": "Poulet nourriture"})

# Part volailles dans la nourriture

calcul_part_poulet_nourriture = pd.merge(pivot_total_nourriture,
                                         pivot_part_poulet_nourriture,
                                         on="Zone",
                                         how="left")
calcul_part_poulet_nourriture["% Volaille / Nourriture"] = round(
    (calcul_part_poulet_nourriture["Poulet nourriture"] /
     calcul_part_poulet_nourriture["Total nourriture"]) * 100, 2)

calcul_part_poulet_nourriture.head()

,Total nourriture,Poulet nourriture,% Volaille / Nourriture
Zone,,,
Afghanistan,12968.0,55.0,0.42
Afrique du Sud,31726.0,2035.0,6.41
Albanie,3631.0,47.0,1.29
Algérie,33056.0,264.0,0.80
Allemagne,77988.0,1609.0,2.06


In [363]:
# Importation totale

total_import = df_dispo_alim_2017.loc[df_dispo_alim_2017["Élément"] ==
                                   "Importations - Quantité"]
pivot_total_import = total_import.pivot_table(index="Zone",
                                              values="Valeur",
                                              aggfunc={"Valeur": numpy.sum})
pivot_total_import = pivot_total_import.rename(
    columns={"Valeur": "Total import"})

# Nourriture volailles

part_poulet_import = df_dispo_alim_2017.loc[
    (df_dispo_alim_2017["Élément"] == "Importations - Quantité")
    & (df_dispo_alim_2017["Produit"] == "Viande de Volailles")]
pivot_part_poulet_import = part_poulet_import.pivot_table(
    index="Zone", values="Valeur", aggfunc={"Valeur": numpy.sum})
pivot_part_poulet_import = pivot_part_poulet_import.rename(
    columns={"Valeur": "Poulet import"})

# Part volailles nourriture

calcul_part_poulet_import = pd.merge(pivot_total_import,
                                     pivot_part_poulet_import,
                                     on="Zone",
                                     how="left")
calcul_part_poulet_import["% Volaille / Import"] = round(
    (calcul_part_poulet_import["Poulet import"] /
     calcul_part_poulet_import["Total import"]) * 100, 2)

calcul_part_poulet_import

,Total import,Poulet import,% Volaille / Import
Zone,,,
Afghanistan,4555.0,29.0,0.64
Afrique du Sud,9020.0,514.0,5.70
Albanie,884.0,38.0,4.30
Algérie,19718.0,2.0,0.01
Allemagne,66550.0,842.0,1.27
...,...,...,...
Émirats arabes unis,14986.0,433.0,2.89
Équateur,2134.0,0.0,0.00
États-Unis d'Amérique,84507.0,123.0,0.15


In [364]:
# On crée un DF qui rassemble part nourriture et import

calcul_part_poulet_nourriture_import =pd.merge(calcul_part_poulet_nourriture, 
                        calcul_part_poulet_import,
                        on='Zone',
                        how='outer'
                        )

calcul_part_poulet_nourriture_import

,Total nourriture,Poulet nourriture,% Volaille / Nourriture,Total import,Poulet import,% Volaille / Import
Zone,,,,,,
Afghanistan,12968.0,55.0,0.42,4555.0,29.0,0.64
Afrique du Sud,31726.0,2035.0,6.41,9020.0,514.0,5.70
Albanie,3631.0,47.0,1.29,884.0,38.0,4.30
Algérie,33056.0,264.0,0.80,19718.0,2.0,0.01
Allemagne,77988.0,1609.0,2.06,66550.0,842.0,1.27
...,...,...,...,...,...,...
Émirats arabes unis,5958.0,412.0,6.92,14986.0,433.0,2.89
Équateur,9465.0,324.0,3.42,2134.0,0.0,0.00
États-Unis d'Amérique,322036.0,18100.0,5.62,84507.0,123.0,0.15


In [365]:
# Dispo alim totale

total_dispo_alim = df_dispo_alim_2017.loc[
    df_dispo_alim_2017["Élément"] ==
    "Disponibilité alimentaire en quantité (kg/personne/an)"]
pivot_total_dispo_alim = total_dispo_alim.pivot_table(
    index="Zone", values="Valeur", aggfunc={"Valeur": numpy.sum})
pivot_total_dispo_alim = pivot_total_dispo_alim.rename(
    columns={"Valeur": "Total dispo alim"})

# Nourriture volailles

part_poulet_dispo_alim = df_dispo_alim_2017.loc[
    (df_dispo_alim_2017["Élément"] ==
     "Disponibilité alimentaire en quantité (kg/personne/an)")
    & (df_dispo_alim_2017["Produit"] == "Viande de Volailles")]
pivot_part_poulet_dispo_alim = part_poulet_dispo_alim.pivot_table(
    index="Zone", values="Valeur", aggfunc={"Valeur": numpy.sum})
pivot_part_poulet_dispo_alim = pivot_part_poulet_dispo_alim.rename(
    columns={"Valeur": "Poulet dispo alim"})

# Part volailles nourriture

calcul_part_poulet_dispo_alim = pd.merge(pivot_total_dispo_alim,
                                         pivot_part_poulet_dispo_alim,
                                         on="Zone",
                                         how="left")
calcul_part_poulet_dispo_alim["% Volaille / Dispo alim (kg/pers/an)"] = round(
    (calcul_part_poulet_dispo_alim["Poulet dispo alim"] /
     calcul_part_poulet_dispo_alim["Total dispo alim"]) * 100, 2)
calcul_part_poulet_dispo_alim

,Total dispo alim,Poulet dispo alim,% Volaille / Dispo alim (kg/pers/an)
Zone,,,
Afghanistan,357.31,1.53,0.43
Afrique du Sud,556.42,35.69,6.41
Albanie,1260.49,16.36,1.30
Algérie,798.71,6.38,0.80
Allemagne,943.24,19.47,2.06
...,...,...,...
Émirats arabes unis,627.67,43.47,6.93
Équateur,563.62,19.31,3.43
États-Unis d'Amérique,990.38,55.68,5.62


Les données issues du DF disponibilité alimentaire sont prêtes pour une future jointure.

## DF pop_2000_2018

In [366]:
# On affiche le DF

df_pop_2000_2018.head()

,Code Domaine,Domaine,Code zone,Zone,Code Élément,Élément,Code Produit,Produit,Code année,Année,Unité,Valeur,Symbole,Description du Symbole,Note
0,OA,Séries temporelles annuelles,2,Afghanistan,511,Population totale,3010,Population-Estimations,2000,2000,1000 personnes,20779.953,X,Sources internationales sûres,NaN
1,OA,Séries temporelles annuelles,2,Afghanistan,511,Population totale,3010,Population-Estimations,2001,2001,1000 personnes,21606.988,X,Sources internationales sûres,NaN
2,OA,Séries temporelles annuelles,2,Afghanistan,511,Population totale,3010,Population-Estimations,2002,2002,1000 personnes,22600.770,X,Sources internationales sûres,NaN
3,OA,Séries temporelles annuelles,2,Afghanistan,511,Population totale,3010,Population-Estimations,2003,2003,1000 personnes,23680.871,X,Sources internationales sûres,NaN
4,OA,Séries temporelles annuelles,2,Afghanistan,511,Population totale,3010,Population-Estimations,2004,2004,1000 personnes,24726.684,X,Sources internationales sûres,NaN


In [367]:
# Les populations s'affichaient avec des décimales, nous les convertissont donc en integer

df_pop_2000_2018["Population totale"] = df_pop_2000_2018["Valeur"] * 1000
df_pop_2000_2018["Population totale"] = df_pop_2000_2018["Population totale"].astype(
    "int64")

In [368]:
# On supprime les variables non nécessaires à l'analyse

pop_2000_2018 = df_pop_2000_2018.drop(columns=[
    "Code Domaine", "Domaine", "Code zone", "Code Élément", "Élément",
    "Code Produit", "Produit", "Code année", "Unité", "Valeur", "Symbole",
    "Description du Symbole", "Note"
])

In [369]:
# On garde uniquement les données de 2017

df_pop_2017 = pop_2000_2018.loc[pop_2000_2018["Année"] == 2017]
df_pop_2017 = df_pop_2017.drop(columns=["Année"])

## DF pib_stab_po_2017

In [370]:
# On affiche le DF

df_pib_stab_pop_2017.head()

,Code zone (ISO3),Zone,Code élément,Élément,Code Produit,Produit,Code année,Année,Valeur
0,AFG,Afghanistan,6126,Valeur,22013,PIB par habitant,2017,2017,2058.40
1,AFG,Afghanistan,6125,Valeur,21032,Stabilité politique,2017,2017,-2.80
2,ZAF,Afrique du Sud,6126,Valeur,22013,PIB par habitant,2017,2017,12701.30
3,ZAF,Afrique du Sud,6125,Valeur,21032,Stabilité politique,2017,2017,-0.28
4,ALB,Albanie,6126,Valeur,22013,PIB par habitant,2017,2017,12771.00


In [371]:
# On crée un pivot table qui affiche PIB/hab et stabilité politique pour chaque pays

PIB_Stabilite_Politique_2017 = df_pib_stab_pop_2017.drop(
    columns=["Code Produit", "Code année", "Année"])

pivot_PIB_Stabilite_Politique_2017 = PIB_Stabilite_Politique_2017.pivot_table(
    index=["Code zone (ISO3)","Zone"], columns="Produit", values="Valeur")
pivot_PIB_Stabilite_Politique_2017.head()

,Produit,PIB par habitant,Stabilité politique
Code zone (ISO3),Zone,,
AFG,Afghanistan,2058.4,-2.80
AGO,Angola,7310.9,-0.33
ALB,Albanie,12771.0,0.38
AND,Andorre,NaN,1.42
ARE,Émirats arabes unis,67183.6,0.62


In [372]:
# On check les valeurs manquantes
nan_pib = pivot_PIB_Stabilite_Politique_2017.loc[pivot_PIB_Stabilite_Politique_2017['PIB par habitant'].isna()]
nan_pib

,Produit,PIB par habitant,Stabilité politique
Code zone (ISO3),Zone,,
AND,Andorre,NaN,1.42
ASM,Samoa américaines,NaN,1.22
CUB,Cuba,NaN,0.65
ERI,Érythrée,NaN,-0.73
GRL,Groenland,NaN,1.92
PRK,République populaire démocratique de Corée,NaN,-0.50
SSD,Soudan du Sud,NaN,-2.45
SYR,République arabe syrienne,NaN,-2.62
TWN,Chine - Taiwan Province de,NaN,0.86


## DF internet_taux

In [373]:
# On affiche le DF

df_internet_taux.head()

,Country Name,Country Code,Indicator Name,Indicator Code,1960,1961,1962,1963,1964,1965,...,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021
0,Aruba,ABW,Utilisateurs d Internet (% de la population),IT.NET.USER.ZS,NaN,NaN,NaN,NaN,NaN,NaN,...,74.000000,78.9,83.78,88.661227,93.542454,97.170000,NaN,NaN,NaN,NaN
1,Afghanistan,AFG,Utilisateurs d Internet (% de la population),IT.NET.USER.ZS,NaN,NaN,NaN,NaN,NaN,NaN,...,5.454545,5.9,7.00,8.260000,11.000000,13.500000,16.8,17.600000,18.400000,NaN
2,Angola,AGO,Utilisateurs d Internet (% de la population),IT.NET.USER.ZS,NaN,NaN,NaN,NaN,NaN,NaN,...,7.700000,13.0,21.40,22.000000,23.200000,26.000000,29.0,32.400000,36.000000,NaN
3,Albanie,ALB,Utilisateurs d Internet (% de la population),IT.NET.USER.ZS,NaN,NaN,NaN,NaN,NaN,NaN,...,49.400000,51.8,54.30,56.900000,59.600000,62.400000,65.4,68.550391,72.237677,79.323718
4,Andorre,AND,Utilisateurs d Internet (% de la population),IT.NET.USER.ZS,NaN,NaN,NaN,NaN,NaN,NaN,...,82.700000,84.4,86.10,87.900000,89.700000,91.567467,NaN,NaN,NaN,NaN


In [374]:
# On crée un df qui contient le pourcentage de personnes ayant accès à internet 

df_internet_taux = df_internet_taux[['Country Name','Country Code','2017']]
df_internet_taux = df_internet_taux.rename(
    columns={
        "Country Name": "Zone",
        "Country Code": "Code zone (ISO3)",
        "2017": "% d'utilisateurs web"
        })
df_internet_taux["% d'utilisateurs web"] = round(
    df_internet_taux["% d'utilisateurs web"], 2)
df_internet_taux.head()

,Zone,Code zone (ISO3),% d'utilisateurs web
0,Aruba,ABW,97.17
1,Afghanistan,AFG,13.50
2,Angola,AGO,26.00
3,Albanie,ALB,62.40
4,Andorre,AND,91.57


## DF revenu_taxe_commerce

In [375]:
df_revenu_taxe_commerce.head()

,Country Name,Country Code,Indicator Name,Indicator Code,1960,1961,1962,1963,1964,1965,...,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021
0,Aruba,ABW,Taxes sur le commerce international (% des rev...,GC.TAX.INTT.RV.ZS,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Afghanistan,AFG,Taxes sur le commerce international (% des rev...,GC.TAX.INTT.RV.ZS,NaN,NaN,NaN,NaN,NaN,NaN,...,4.224652,3.703643,3.806549,4.570932,6.381247,7.422589,NaN,NaN,NaN,NaN
2,Angola,AGO,Taxes sur le commerce international (% des rev...,GC.TAX.INTT.RV.ZS,NaN,NaN,NaN,NaN,NaN,NaN,...,2.513808,2.640707,4.115002,4.980515,3.436295,3.628489,2.706723,2.872464,NaN,NaN
3,Albanie,ALB,Taxes sur le commerce international (% des rev...,GC.TAX.INTT.RV.ZS,NaN,NaN,NaN,NaN,NaN,NaN,...,3.890822,3.564892,2.902073,2.910302,1.658020,1.586931,1.469280,1.500913,1.566248,NaN
4,Andorre,AND,Taxes sur le commerce international (% des rev...,GC.TAX.INTT.RV.ZS,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [376]:
# On supprime les colonnes inutiles 

df_revenu_taxe_commerce = df_revenu_taxe_commerce.drop(columns=[
    "Indicator Name", "Indicator Code", "1960", "1961", "1962", "1963", "1964",
    "1965", "1966", "1967", "1968", "1969", "1970", "1971", "1972", "1973",
    "1974", "1975", "1976", "1977", "1978", "1979", "1980", "1981", "1982",
    "1983", "1984", "1985", "1986", "1987", "1988", "1989", "1990", "1991",
    "1992", "1993", "1994", "1995", "1996", "1997", "1998", "1999", "2000",
    "2001", "2002", "2003", "2004", "2005", "2006", "2007", "2008", "2009",
    "2010", "2011", "2012", "2013", "2014", "2015", "2016", "2018", "2019",
    "2020", "2021",
    ])

In [377]:
# On renomme les colonnes et on arrondit

df_revenu_taxe_commerce = df_revenu_taxe_commerce.rename(
    columns={
        "Country Name": "Zone",
        "Country Code": "Code zone (ISO3)",
        "2017": "% des revenus"
    })
df_revenu_taxe_commerce["% des revenus"] = round(
    df_revenu_taxe_commerce["% des revenus"], 2)
df_revenu_taxe_commerce

,Zone,Code zone (ISO3),% des revenus
0,Aruba,ABW,NaN
1,Afghanistan,AFG,7.42
2,Angola,AGO,3.63
3,Albanie,ALB,1.59
4,Andorre,AND,NaN
...,...,...,...
213,Kosovo,XKX,NaN
214,"Yémen, Rép. du",YEM,NaN
215,Afrique du Sud,ZAF,3.17
216,Zambie,ZMB,5.44


## DF distance

In [378]:
# On crée un DF qui contient uniquement les distances depuis la France
df_france_distances = pd.DataFrame(df_distance['FRA'].values, columns=['Distance from France'], index=df_distance.index)
df_france_distances = df_france_distances.rename_axis('Code zone (ISO3)').reset_index()
df_france_distances['Distance from France'] = round(df_france_distances['Distance from France'],0)
df_france_distances

,Code zone (ISO3),Distance from France
0,ABW,7677.0
1,AFG,5594.0
2,AGO,6479.0
3,AIA,6711.0
4,ALB,1604.0
...,...,...
230,WSM,16083.0
231,YEM,5312.0
232,ZAF,9307.0
233,ZMB,7565.0


## Jointures 

### Création du DF volaille

In [379]:
# On effectue une jointure pour inclure tous nos éléments liés aux volailles

df_volaille = pd.merge(pd.merge(calcul_part_dispo_prod,
                                            calcul_part_poulet_nourriture_import,
                                            on="Zone",
                                            how="outer"),
                                   calcul_part_poulet_dispo_alim,
                                   on="Zone",
                                   how="outer")
df_volaille.head()

,Total disponibilité intérieure,Poulet dispo intérieur,% Volaille / Dispo intérieur,Total production,Poulet production,% Volaille / Production,Total nourriture,Poulet nourriture,% Volaille / Nourriture,Total import,Poulet import,% Volaille / Import,Total dispo alim,Poulet dispo alim,% Volaille / Dispo alim (kg/pers/an)
Zone,,,,,,,,,,,,,,,
Afghanistan,15139.0,57.0,0.38,11609.0,28.0,0.24,12968.0,55.0,0.42,4555.0,29.0,0.64,357.31,1.53,0.43
Afrique du Sud,66840.0,2118.0,3.17,68455.0,1667.0,2.44,31726.0,2035.0,6.41,9020.0,514.0,5.70,556.42,35.69,6.41
Albanie,4879.0,47.0,0.96,4231.0,13.0,0.31,3631.0,47.0,1.29,884.0,38.0,4.30,1260.49,16.36,1.30
Algérie,45365.0,277.0,0.61,26763.0,275.0,1.03,33056.0,264.0,0.80,19718.0,2.0,0.01,798.71,6.38,0.80
Allemagne,174960.0,1739.0,0.99,167828.0,1514.0,0.90,77988.0,1609.0,2.06,66550.0,842.0,1.27,943.24,19.47,2.06


### Jointure des données de population

In [380]:
# Reset de l'index pour avoir Zone en colonne pour la jointure

df_volaille = df_volaille.reset_index()

# Jointure 

df_volaille_pop = pd.merge(df_volaille, df_pop_2017, on="Zone")
df_volaille_pop.head()

,Zone,Total disponibilité intérieure,Poulet dispo intérieur,% Volaille / Dispo intérieur,Total production,Poulet production,% Volaille / Production,Total nourriture,Poulet nourriture,% Volaille / Nourriture,Total import,Poulet import,% Volaille / Import,Total dispo alim,Poulet dispo alim,% Volaille / Dispo alim (kg/pers/an),Population totale
0,Afghanistan,15139.0,57.0,0.38,11609.0,28.0,0.24,12968.0,55.0,0.42,4555.0,29.0,0.64,357.31,1.53,0.43,36296113
1,Afrique du Sud,66840.0,2118.0,3.17,68455.0,1667.0,2.44,31726.0,2035.0,6.41,9020.0,514.0,5.70,556.42,35.69,6.41,57009756
2,Albanie,4879.0,47.0,0.96,4231.0,13.0,0.31,3631.0,47.0,1.29,884.0,38.0,4.30,1260.49,16.36,1.30,2884169
3,Algérie,45365.0,277.0,0.61,26763.0,275.0,1.03,33056.0,264.0,0.80,19718.0,2.0,0.01,798.71,6.38,0.80,41389189
4,Allemagne,174960.0,1739.0,0.99,167828.0,1514.0,0.90,77988.0,1609.0,2.06,66550.0,842.0,1.27,943.24,19.47,2.06,82658409


### Jointure des données sur le PIB et la stabilité politique

In [381]:
# On reset l'index pour la jointure

pivot_PIB_Stabilite_Politique_2017 = pivot_PIB_Stabilite_Politique_2017.reset_index()

# On effectue la jointure 

df_volaille_pop_pib_stab = pd.merge(df_volaille_pop,
                               pivot_PIB_Stabilite_Politique_2017,
                               on="Zone",
                               how="left")
df_volaille_pop_pib_stab.head()

,Zone,Total disponibilité intérieure,Poulet dispo intérieur,% Volaille / Dispo intérieur,Total production,Poulet production,% Volaille / Production,Total nourriture,Poulet nourriture,% Volaille / Nourriture,Total import,Poulet import,% Volaille / Import,Total dispo alim,Poulet dispo alim,% Volaille / Dispo alim (kg/pers/an),Population totale,Code zone (ISO3),PIB par habitant,Stabilité politique
0,Afghanistan,15139.0,57.0,0.38,11609.0,28.0,0.24,12968.0,55.0,0.42,4555.0,29.0,0.64,357.31,1.53,0.43,36296113,AFG,2058.4,-2.80
1,Afrique du Sud,66840.0,2118.0,3.17,68455.0,1667.0,2.44,31726.0,2035.0,6.41,9020.0,514.0,5.70,556.42,35.69,6.41,57009756,ZAF,12701.3,-0.28
2,Albanie,4879.0,47.0,0.96,4231.0,13.0,0.31,3631.0,47.0,1.29,884.0,38.0,4.30,1260.49,16.36,1.30,2884169,ALB,12771.0,0.38
3,Algérie,45365.0,277.0,0.61,26763.0,275.0,1.03,33056.0,264.0,0.80,19718.0,2.0,0.01,798.71,6.38,0.80,41389189,DZA,11737.4,-0.92
4,Allemagne,174960.0,1739.0,0.99,167828.0,1514.0,0.90,77988.0,1609.0,2.06,66550.0,842.0,1.27,943.24,19.47,2.06,82658409,DEU,52952.9,0.59


### Jointure des données Internet et Taxes

In [382]:
df_volaille_pop_pib_stab_int_tax = pd.merge(pd.merge(df_volaille_pop_pib_stab,
                                        df_internet_taux,
                                        on="Code zone (ISO3)",
                                        how="left"),
                               df_revenu_taxe_commerce,
                               on="Code zone (ISO3)",
                               how="left")

df_volaille_pop_pib_stab_int_tax = df_volaille_pop_pib_stab_int_tax.drop(['Zone_x', 'Zone_y'], axis=1)
df_volaille_pop_pib_stab_int_tax

,Total disponibilité intérieure,Poulet dispo intérieur,% Volaille / Dispo intérieur,Total production,Poulet production,% Volaille / Production,Total nourriture,Poulet nourriture,% Volaille / Nourriture,Total import,...,Total dispo alim,Poulet dispo alim,% Volaille / Dispo alim (kg/pers/an),Population totale,Code zone (ISO3),PIB par habitant,Stabilité politique,% d'utilisateurs web,Zone,% des revenus
0,15139.0,57.0,0.38,11609.0,28.0,0.24,12968.0,55.0,0.42,4555.0,...,357.31,1.53,0.43,36296113,AFG,2058.4,-2.80,13.50,Afghanistan,7.42
1,66840.0,2118.0,3.17,68455.0,1667.0,2.44,31726.0,2035.0,6.41,9020.0,...,556.42,35.69,6.41,57009756,ZAF,12701.3,-0.28,56.17,Afrique du Sud,3.17
2,4879.0,47.0,0.96,4231.0,13.0,0.31,3631.0,47.0,1.29,884.0,...,1260.49,16.36,1.30,2884169,ALB,12771.0,0.38,62.40,Albanie,1.59
3,45365.0,277.0,0.61,26763.0,275.0,1.03,33056.0,264.0,0.80,19718.0,...,798.71,6.38,0.80,41389189,DZA,11737.4,-0.92,47.69,Algérie,NaN
4,174960.0,1739.0,0.99,167828.0,1514.0,0.90,77988.0,1609.0,2.06,66550.0,...,943.24,19.47,2.06,82658409,DEU,52952.9,0.59,84.39,Allemagne,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
169,11795.0,412.0,3.49,1793.0,48.0,2.68,5958.0,412.0,6.92,14986.0,...,627.67,43.47,6.93,9487203,ARE,67183.6,0.62,94.82,Émirats arabes unis,0.08
170,23810.0,341.0,1.43,30843.0,340.0,1.10,9465.0,324.0,3.42,2134.0,...,563.62,19.31,3.43,16785361,ECU,11617.9,-0.07,55.80,Équateur,NaN
171,849821.0,18266.0,2.15,974034.0,21914.0,2.25,322036.0,18100.0,5.62,84507.0,...,990.38,55.68,5.62,325084756,USA,60109.7,0.34,87.27,États-Unis,1.01
172,55860.0,14.0,0.03,52164.0,14.0,0.03,38502.0,14.0,0.04,2967.0,...,361.77,0.13,0.04,106399924,ETH,2021.6,-1.68,18.62,Éthiopie,18.74


### Jointure des données de distance

In [383]:
df_final = pd.merge(df_volaille_pop_pib_stab_int_tax,
                                            df_france_distances,
                                            on="Code zone (ISO3)",
                                            how='left')

In [384]:
df_final = df_final.drop(columns=[
    "Total disponibilité intérieure", "Poulet dispo intérieur", "Total production",
    "Poulet production", "Total nourriture", "Poulet nourriture",
    "Total import", "Poulet import", "Total dispo alim", "Poulet dispo alim"
])
df_final

,% Volaille / Dispo intérieur,% Volaille / Production,% Volaille / Nourriture,% Volaille / Import,% Volaille / Dispo alim (kg/pers/an),Population totale,Code zone (ISO3),PIB par habitant,Stabilité politique,% d'utilisateurs web,Zone,% des revenus,Distance from France
0,0.38,0.24,0.42,0.64,0.43,36296113,AFG,2058.4,-2.80,13.50,Afghanistan,7.42,5594.0
1,3.17,2.44,6.41,5.70,6.41,57009756,ZAF,12701.3,-0.28,56.17,Afrique du Sud,3.17,9307.0
2,0.96,0.31,1.29,4.30,1.30,2884169,ALB,12771.0,0.38,62.40,Albanie,1.59,1604.0
3,0.61,1.03,0.80,0.01,0.80,41389189,DZA,11737.4,-0.92,47.69,Algérie,NaN,1345.0
4,0.99,0.90,2.06,1.27,2.06,82658409,DEU,52952.9,0.59,84.39,Allemagne,0.00,881.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
169,3.49,2.68,6.92,2.89,6.93,9487203,ARE,67183.6,0.62,94.82,Émirats arabes unis,0.08,5252.0
170,1.43,1.10,3.42,0.00,3.43,16785361,ECU,11617.9,-0.07,55.80,Équateur,NaN,9626.0
171,2.15,2.25,5.62,0.15,5.62,325084756,USA,60109.7,0.34,87.27,États-Unis,1.01,5853.0
172,0.03,0.03,0.04,0.03,0.04,106399924,ETH,2021.6,-1.68,18.62,Éthiopie,18.74,5566.0


In [385]:
df_final = df_final.reindex(columns=[
    "Zone", "Code zone (ISO3)", "Population totale", "PIB par habitant",
    "Stabilité politique", "% Volaille / Dispo intérieur",
    "% Volaille / Production", "% Volaille / Nourriture",
    "% Volaille / Import", "% Volaille / Dispo alim (kg/pers/an)",
    "% d'utilisateurs web", "% des revenus", "Distance from France"
])
df_final = df_final.rename(
    columns={
        "Code zone (ISO3)": "ISO3",
        "% des revenus": "% des revenus issus des taxes"
    })
df_final

,Zone,ISO3,Population totale,PIB par habitant,Stabilité politique,% Volaille / Dispo intérieur,% Volaille / Production,% Volaille / Nourriture,% Volaille / Import,% Volaille / Dispo alim (kg/pers/an),% d'utilisateurs web,% des revenus issus des taxes,Distance from France
0,Afghanistan,AFG,36296113,2058.4,-2.80,0.38,0.24,0.42,0.64,0.43,13.50,7.42,5594.0
1,Afrique du Sud,ZAF,57009756,12701.3,-0.28,3.17,2.44,6.41,5.70,6.41,56.17,3.17,9307.0
2,Albanie,ALB,2884169,12771.0,0.38,0.96,0.31,1.29,4.30,1.30,62.40,1.59,1604.0
3,Algérie,DZA,41389189,11737.4,-0.92,0.61,1.03,0.80,0.01,0.80,47.69,NaN,1345.0
4,Allemagne,DEU,82658409,52952.9,0.59,0.99,0.90,2.06,1.27,2.06,84.39,0.00,881.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
169,Émirats arabes unis,ARE,9487203,67183.6,0.62,3.49,2.68,6.92,2.89,6.93,94.82,0.08,5252.0
170,Équateur,ECU,16785361,11617.9,-0.07,1.43,1.10,3.42,0.00,3.43,55.80,NaN,9626.0
171,États-Unis,USA,325084756,60109.7,0.34,2.15,2.25,5.62,0.15,5.62,87.27,1.01,5853.0
172,Éthiopie,ETH,106399924,2021.6,-1.68,0.03,0.03,0.04,0.03,0.04,18.62,18.74,5566.0


In [386]:
# Suppression des pays possédant des valeurs nulles pour rendre fiable le PESTEL

df_final = df_final.dropna()


## Analyse univariée

In [387]:
# On indexe le DF sur "Zone"

df_final = df_final.set_index("Zone")

# On supprime la colonne ISO3 pour conserver uniquement les variables quanti

df_final_quanti = df_final.drop(columns="ISO3")
df_final_quanti

,Population totale,PIB par habitant,Stabilité politique,% Volaille / Dispo intérieur,% Volaille / Production,% Volaille / Nourriture,% Volaille / Import,% Volaille / Dispo alim (kg/pers/an),% d'utilisateurs web,% des revenus issus des taxes,Distance from France
Zone,,,,,,,,,,,
Afghanistan,36296113,2058.4,-2.80,0.38,0.24,0.42,0.64,0.43,13.50,7.42,5594.0
Afrique du Sud,57009756,12701.3,-0.28,3.17,2.44,6.41,5.70,6.41,56.17,3.17,9307.0
Albanie,2884169,12771.0,0.38,0.96,0.31,1.29,4.30,1.30,62.40,1.59,1604.0
Allemagne,82658409,52952.9,0.59,0.99,0.90,2.06,1.27,2.06,84.39,0.00,881.0
Angola,29816766,7310.9,-0.33,0.99,0.15,1.85,6.67,1.85,26.00,3.63,6479.0
...,...,...,...,...,...,...,...,...,...,...,...
Zimbabwe,14236595,3274.6,-0.71,0.83,0.91,1.39,0.56,1.38,24.40,7.74,7914.0
Émirats arabes unis,9487203,67183.6,0.62,3.49,2.68,6.92,2.89,6.93,94.82,0.08,5252.0
États-Unis,325084756,60109.7,0.34,2.15,2.25,5.62,0.15,5.62,87.27,1.01,5853.0


### Répartition et outliers

#### Boxplots

In [388]:
# Boxplot de distribution de la population totale par pays 

fig = go.Figure()

fig.add_box(x=df_final_quanti['Population totale'], name='Population totale', orientation='h')

fig.update_layout(
    title="Distribution de la population",
)

fig.show()

In [389]:
# Boxplot de distribution du PIB/habitant

fig = go.Figure()

fig.add_box(y=df_final_quanti['PIB par habitant'], name='PIB par habitant')

fig.update_layout(
    title="Distribution du PIB par habitant"
)

fig.show()

In [390]:
# Boxplots pour les variables liées à la volaille

fig = go.Figure()

fig.add_box(y=df_final_quanti['% Volaille / Nourriture'], name='% Volaille / Nourriture')
fig.add_box(y=df_final_quanti['% Volaille / Import'], name='% Volaille / Import')
fig.add_box(y=df_final_quanti['% Volaille / Dispo intérieur'], name='% Volaille / Dispo intérieur')
fig.add_box(y=df_final_quanti['% Volaille / Production'], name='% Volaille / Production')
fig.add_box(y=df_final_quanti['% Volaille / Dispo alim (kg/pers/an)'], name='% Volaille / Dispo alim (kg/pers/an)')

fig.update_layout(
    height=800, 
    width=1200,
    title="Distribution des variables liées à la volaille"
)

fig.show()

In [391]:
# Boxplot de distribution du taux d'utilisateurs web par pays

fig = go.Figure()

fig.add_box(x=df_final_quanti["% d'utilisateurs web"], name="% d'utilisateurs web", orientation='h', boxpoints='outliers')

fig.update_layout(
    title="Distribution du taux d'utilisateurs web",
)

fig.show()

In [392]:
# Distribution du taux des revenus issus des taxes

fig = go.Figure()

fig.add_box(x=df_final_quanti['% des revenus issus des taxes'], name='% des revenus issus des taxes', orientation='h')

fig.update_layout(
    title="% des revenus issus des taxes",
)

fig.show()

In [393]:
# Distribution de la distance par rapport a la France

fig = go.Figure()

fig.add_box(x=df_final_quanti['Distance from France'], name='Distance from France', orientation='h')

fig.update_layout(
    title="Distance from France",
)

fig.show()

#### Histogrammes

In [394]:
# On crée des histogrammes pour visualiser la répartition de chaque variable 

for col in df_final_quanti.columns:
    fig = px.histogram(df_final_quanti, x=col)
    fig.show()

### Statistiques descriptives

In [395]:
df_final_quanti.describe()

,Population totale,PIB par habitant,Stabilité politique,% Volaille / Dispo intérieur,% Volaille / Production,% Volaille / Nourriture,% Volaille / Import,% Volaille / Dispo alim (kg/pers/an),% d'utilisateurs web,% des revenus issus des taxes,Distance from France
count,1.230000e+02,123.000000,123.000000,123.000000,123.000000,123.000000,123.000000,123.000000,123.000000,123.000000,123.000000
mean,3.805179e+07,22764.709756,0.040732,1.608293,1.449837,2.998049,2.248049,2.964472,56.894959,6.166341,5808.000000
std,1.277747e+08,22084.905803,0.855681,1.596280,2.362640,2.512582,3.564180,2.402368,27.021813,7.322121,4208.075254
min,5.204500e+04,1283.700000,-2.800000,0.030000,0.000000,0.040000,0.000000,0.040000,7.000000,-0.020000,0.000000
25%,2.932820e+06,6093.450000,-0.515000,0.680000,0.310000,1.280000,0.245000,1.285000,32.620000,0.150000,1869.500000
50%,9.785843e+06,14945.700000,0.090000,1.110000,0.870000,2.240000,1.010000,2.230000,63.410000,3.240000,5566.000000
75%,3.127447e+07,33403.300000,0.735000,1.910000,1.660000,4.010000,2.680000,4.010000,80.125000,9.815000,8702.500000
max,1.338677e+09,126184.000000,1.590000,8.510000,18.370000,13.040000,24.290000,10.990000,98.260000,32.480000,18540.000000


## Analyse multivariée

### Matrice de corrélation 



In [396]:
corr_matrix = df_final_quanti.corr()
fig = go.Figure(data=go.Heatmap(z=corr_matrix.values, 
                                 x=corr_matrix.columns, 
                                 y=corr_matrix.columns, 
                                 colorscale="Reds"))

fig.update_layout(
    width=800, 
    height=800,  
    title="Matrice de corrélation",
    xaxis_title="Variables",
    yaxis_title="Variables"
)

fig.show()

Il y a un certain nombre de variables qui concernent les produits volaillers et on constate que certaines d'entre elles sont plutôt fortement corrélées. Ainsi, nous réduirons le nombre de variables que nous conserverons pour notre analyse.

In [397]:
# Suppression des variables non utilisées 

df_final_quanti = df_final_quanti.drop(columns=[
    "% Volaille / Dispo intérieur", "% Volaille / Production",
    "% Volaille / Dispo alim (kg/pers/an)"
])

df_final_quanti

,Population totale,PIB par habitant,Stabilité politique,% Volaille / Nourriture,% Volaille / Import,% d'utilisateurs web,% des revenus issus des taxes,Distance from France
Zone,,,,,,,,
Afghanistan,36296113,2058.4,-2.80,0.42,0.64,13.50,7.42,5594.0
Afrique du Sud,57009756,12701.3,-0.28,6.41,5.70,56.17,3.17,9307.0
Albanie,2884169,12771.0,0.38,1.29,4.30,62.40,1.59,1604.0
Allemagne,82658409,52952.9,0.59,2.06,1.27,84.39,0.00,881.0
Angola,29816766,7310.9,-0.33,1.85,6.67,26.00,3.63,6479.0
...,...,...,...,...,...,...,...,...
Zimbabwe,14236595,3274.6,-0.71,1.39,0.56,24.40,7.74,7914.0
Émirats arabes unis,9487203,67183.6,0.62,6.92,2.89,94.82,0.08,5252.0
États-Unis,325084756,60109.7,0.34,5.62,0.15,87.27,1.01,5853.0


In [398]:
# On exporte le DF au format CSV 

df_final_quanti.to_csv('df_final_quanti.csv')


Le DF est prêt pour le clustering qui sera effectué dans le second notebook.